# Simple script to Create a Crosstab & the KPI File for Dayforce.

In [1]:
import sys,os
from pathlib import Path
from time import sleep

sys.path.append(
    os.path.abspath(
        r"S:\Data\Stores Payroll\FY21\99_Master Scripts (DO NOT EDIT)\dB_Connector"))

from connector import *

sys.path.append(
    os.path.abspath(
        r"S:\Data\Stores Payroll\FY21\99_Master Scripts (DO NOT EDIT)\common_functions"))

from halfords_functions import newest, halfords_week

import pandas as pd
from datetime import datetime
import shutil
import numpy as np

In [2]:
# fy21 dates.

dates = pd.read_sql("SELECT * from fy21_calendar", con=engine)

# structure tab wtih shop names etc.

structure = pd.read_sql("SELECT Shop as store from structure_tab", engine)


In [3]:
file_name,week_,day_ = halfords_week(dates)
print(f"We are {week_} weeks away from FY21")

As today is 2019-09-26 we haven't started FY21 so we will be using a psuedo week number which is calculated
from the distance of weeks from FY21
We are -27 weeks away from FY21


In [4]:
# Create an empty dataframe with a value for every week of the year.

structure['Type'] = 'Extra'
structure['WeekYear'] = 2020001
structure = pd.concat([structure]*52)

structure['WeekYear'] = structure['WeekYear'].add(structure.groupby(['store']).cumcount())

In [5]:
# Read in the Structure Tab as an aggregate query.

xh20 = pd.read_sql(
    "SELECT shop as store, sum(hours) as Hours, [Week Number] from extraHoursDetails group by [Week Number], Shop",
    engine,
)

xh20["Week Number"] = xh20["Week Number"].astype(str).str.zfill(3)


xh20["WeekYear"] = "2020" + xh20["Week Number"]


xh20["WeekYear"] = xh20["WeekYear"].astype(int)






In [6]:
structure['store'] = structure['store'].astype(int)

In [7]:
finalDF = (
    pd.merge(
        structure,
        xh20[["store", "WeekYear", "Hours"]],
        on=["store", "WeekYear"],
        how="left",
    )
    .fillna(0)
    .sort_values(["store", "WeekYear"])
)

finalDF = finalDF[["Type", "WeekYear", "store", "Hours"]].reset_index(drop=True)




[Y] or [N]y


saving file to C:\Users\hal01845\OneDrive - HALFORDSGROUP\Master Backup of Umar's Scripts DO NOT EDIT\FY21\Weekly Tasks


In [13]:
os.chdir(r'S:\Data\Stores Payroll\FY21\02_Weekly Tasks\Extra Hours\dayforce_kpi_files')

finalDF.to_csv(f'{file_name}_dayforce_kpi_unprocessed.csv',index=False,header=None)

os.chdir(r"G:\MFRAME\FTPDATA\Dayforce\PRD\KPIHours")


print(f"Do you wish to save the KPI file to {os.getcwd()}")

cmd = input("[Y] or [N]")

while True:
    if cmd.lower().strip() == 'y':
        print(f"saving file to {os.getcwd()}")
        the_path = '"G:\MFRAME\FTPDATA\Dayforce\PRD\KPIHours'
        print(f"file saved to : {the_path} please email the IT helpdesk asking them to process the file")
        break
    elif cmd.lower().strip() == 'n':
        print("Saving file to \\extra hours\dayforce_kpi_files only")
        break
    else:
        print("Please enter Y or N")




Do you wish to save the KPI file to G:\MFRAME\FTPDATA\Dayforce\PRD\KPIHours
[Y] or [N]y
saving file to G:\MFRAME\FTPDATA\Dayforce\PRD\KPIHours
file saved to : "G:\MFRAME\FTPDATA\Dayforce\PRD\KPIHours please email the IT helpdesk asking them to process the file


In [62]:
xh20 = "SELECT * from extraHoursDetails"

xh20 = pd.read_sql(xh20,engine)

st = pd.read_sql("SELECT Shop as Store from structure_tab", engine)

## Create a Week Column and set to Week 1 ## 
st['Week'] = 2101


st = pd.concat([st]*52)

st['Store'] = st['Store'].astype(int).astype(str).str.zfill(4)

st['Week'] = st.Week.add(st.groupby(['Store']).cumcount())

xh20ct = pd.merge(st,xh20[['Store','Week','Hours']],on=['Store','Week'],how='left').fillna(0)

xh20ct = pd.crosstab(xh20ct['Store'],xh20ct['Week'],xh20ct['Hours'],aggfunc='sum').reset_index()

xh20ct.columns = xh20ct.columns.astype(str)

xh20ct.to_sql('extraHours',con=engine,schema='dbo',if_exists='replace',index=False,dtype={'Store' : sa.types.VARCHAR(length=255)})

print("Crosstab Updated.")

Crosstab Updated.


Store                object
Shop                float64
Week                  int64
Hours               float64
Reason               object
CostCentre           object
Type                 object
Rate                float64
Owner                object
BusinessFunction     object
EmployeeNumber      float64
Week Number           int64
dtype: object